In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from geonamescache import GeonamesCache
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap

filename = 'wbdata2.csv'
shapefile = 'shp/countries/ne_10m_admin_0_countries_lakes'
num_colors = 9
yearrange = ['1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']

cols = ['Country Name', 'Country Code', year]
title = 'Youth Unemployment in {}, (Percentage of Total Labor Force)'.format(year)

description = '''
Youth unemployment refers to the share of the labor force ages 15-24 without work but available for and seeking employment. Source: worldbank.org'''.strip()

NameError: name 'year' is not defined

In [ ]:
gc = GeonamesCache()
iso3_codes = list(gc.get_dataset_by_key(gc.get_countries(), 'iso3').keys())

In [ ]:
df = pd.read_csv(filename, skiprows=4, usecols=cols)
df.set_index('Country Code', inplace=True)
df = df.ix[iso3_codes].dropna() # Filter out non-countries and missing values.

In [ ]:
values = df[year]
cm = plt.get_cmap('Reds')
scheme = cm(1.*np.arange(num_colors)/num_colors)
bins = np.linspace(values.min(), values.max(), num_colors)
df['bin'] = np.digitize(values, bins) - 1
df.sort_values('bin', ascending=False).head(5)

In [ ]:
for year in yearrange:

    cols = ['Country Name', 'Country Code', year]
    title = 'Youth Unemployment in {}, (Percentage of Total Labor Force)'.format(year)

    description = '''
    Youth unemployment refers to the share of the labor force ages 15-24 without work but available for and seeking employment. Source: worldbank.org'''.strip()

    df = pd.read_csv(filename, skiprows=4, usecols=cols)
    df.set_index('Country Code', inplace=True)
    df = df.ix[iso3_codes].dropna() # Filter out non-countries and missing values.

    values = df[year]
    cm = plt.get_cmap('Reds')
    scheme = cm(1.*np.arange(num_colors)/num_colors)
    bins = np.linspace(values.min(), values.max(), num_colors)
    df['bin'] = np.digitize(values, bins) - 1
    df.sort_values('bin', ascending=False).head(5)


    mpl.style.use('seaborn-white')
    fig = plt.figure(figsize=(22, 12))

    ax = fig.add_subplot(111, axisbg='w', frame_on=False)
    fig.suptitle('Youth Unemployment Rate by Country in {}'.format(year), fontsize=30, y=.95)

    m = Basemap(lon_0=0, projection='robin')
    m.drawmapboundary(color='#ffffff')

    m.readshapefile(shapefile, 'units', linewidth=0)
    for info, shape in zip(m.units_info, m.units):
        iso3 = info['ADM0_A3']
        if iso3 not in df.index:
            color = '#dddddd'
        else:
            color = scheme[df.ix[iso3]['bin']]

        patches = [Polygon(np.array(shape), True)]
        pc = PatchCollection(patches, edgecolor='#7c7c7c', lw=.2, alpha=1.)
        pc.set_facecolor(color)
        ax.add_collection(pc)

    # Cover up Antarctica so legend can be placed over it.
    ax.axhspan(0, 1000 * 1800, facecolor='w', edgecolor='w', zorder=2)

    # Draw color legend.
    ax_legend = fig.add_axes([0.35, 0.14, 0.3, 0.03], zorder=3)
    cmap = mpl.colors.ListedColormap(scheme)
    cb = mpl.colorbar.ColorbarBase(ax_legend, cmap=cmap, ticks=bins, boundaries=bins, orientation='horizontal')
    cb.ax.set_xticklabels([str(round(i, 1)) for i in bins])
    cb.set_clim(0, 80)

    # Set the map footer.
    plt.annotate(description.decode('utf-8'), xy=(-.5, -2), size=14, xycoords='axes fraction')

    plt.savefig('unemp_' + year + '.png', bbox_inches='tight', pad_inches=.2, dpi=100)
    plt.clf()
    pass

In [ ]:
from images2gif import writeGif
from PIL import Image
import os

file_names = sorted((fn for fn in os.listdir('.') if fn.endswith('.png')))

images = [Image.open(fn) for fn in file_names]

filename = "youthunemp.GIF"
writeGif(filename, images, duration=1)